# Polynomial class


**Objective:** Implement a Python class to represent and manipulate polynomials.

### Requirements:

1. **Polynomial class**: 
    - Create a Polynomial class that accepts coefficients as a list, representing a polynomial. For example, the list `[1, -3, 2]` represents the polynomial $ x^2 - 3x + 2$
    - The class must be callable. Calling a Polynomial object requires a float value `x` and returns a float corresponding to the value of the polynomial at `x`
    - Implement a __repr__ method for suitable printing.
    - Implement addition and subtraction of two polynomials using `+` and `-` operators. 
    - Implement multiplication of two polynomials using the `*` operator.
    - Create a `class_method` called `from_roots` to create a polynomial given its roots. (see `numpy.poly`)

2. **Differentiate**: create a function called `differentiate` that takes a `Polynomial` as input and returns its derivative (as a `Polynomial` object)

3. **Linear and quadratic polynomials**: create two classes called **LinearPolynomial** and **QuadraticPolynomial**, inheriting from `Polynomial`.
    - Both override the init method to ensure that the `list` given as input contains the correct number of coefficients.
    - The linear polynomial also contains a method called `slope` that returns its linear coefficient
    - The quadratic polynomial adds 2 methods:
        - `discriminant` computes its discriminant
        - `roots` computes the roots using the quadratic formula (results can be complex-valued)

4. **(Additional) the logging operator**: create a logger decorator that is called everytime an addition or multiplication is performed. It outputs the name and result of the operation.

In [1]:
import numpy as np

def log_operation(func):
    def wrapper(p1, p2):
        print(f"We are about to call {func.__name__}.")
        result = func(p1, p2)
        print(f"The result is {result}")
        return result
    return wrapper

def differentiate(polynomial):
    result = [polynomial.coefficients[i] * (len(polynomial.coefficients) - i - 1)
              for i in range(len(polynomial.coefficients) - 1)]
    return Polynomial(result)

def integrate(polynomial):
    result = [polynomial.coefficients[i] / (len(polynomial.coefficients) - i)
              for i in range(len(polynomial.coefficients))]
    result.append(0)  # Add integration constant (0 by default)
    return Polynomial(result)

class Polynomial:
    def __init__(self, coefficients):
        self.coefficients = coefficients

    def __call__(self, x):
        return np.polyval(self.coefficients, x)

    def call_2(self, x):
        result = 0
        max_degree = len(self.coefficients) - 1
        for i, coefficient in enumerate(self.coefficients):
            current_degree = max_degree - i
            result += coefficient * (x ** current_degree)
        return result

    def __repr__(self):
        eq = []
        max_degree = len(self.coefficients) - 1
        for i, coefficient in enumerate(self.coefficients):
            current_degree = max_degree - i
            if coefficient != 0:
                if current_degree == 0:
                    term = f"{coefficient}"
                elif current_degree == 1:
                    term = f"{coefficient}x"
                else:
                    term = f"{coefficient}x^{current_degree}"
                eq.append(term)
        polynomial_str = "+".join(eq)
        return polynomial_str.replace("+-", "-")

    @log_operation
    def __add__(self, p2):
        return np.polyadd(self.coefficients, p2.coefficients)

    @log_operation
    def add_2(self, p2):
        # Handle different lengths by padding with zeros
        coeff1 = [0] * (len(p2.coefficients) - len(self.coefficients)) + self.coefficients
        coeff2 = [0] * (len(self.coefficients) - len(p2.coefficients)) + p2.coefficients
        result = [coeff1[i] + coeff2[i] for i in range(len(coeff1))]
        return Polynomial(result)

    def __sub__(self, p2):
        return np.polysub(self.coefficients, p2.coefficients)

    def sub_2(self, p2):
        # Handle different lengths by padding with zeros
        coeff1 = [0] * (len(p2.coefficients) - len(self.coefficients)) + self.coefficients
        coeff2 = [0] * (len(self.coefficients) - len(p2.coefficients)) + p2.coefficients
        result = [coeff1[i] - coeff2[i] for i in range(len(coeff1))]
        return Polynomial(result)

    @log_operation
    def __mul__(self, p2):
        return np.polymul(self.coefficients, p2.coefficients)

    @log_operation
    def mul_2(self, p2):
        result = [0] * (len(self.coefficients) + len(p2.coefficients) - 1)
        for i in range(len(self.coefficients)):
            for j in range(len(p2.coefficients)):
                result[i + j] += self.coefficients[i] * p2.coefficients[j]
        return Polynomial(result)

    def get_roots(self):
        return np.roots(self.coefficients)

    def from_roots(self, roots):
        return Polynomial(np.poly(roots))

class LinearPolynomial(Polynomial):
    def __init__(self, coefficients):
        super().__init__(coefficients)
        if len(coefficients) != 2:
            raise ValueError
    
    def slope(self):
        return self.coefficients[1]

class QuadraticPolynomial(Polynomial):
    def __init__(self, coefficients):
        super().__init__(coefficients)
        if len(coefficients) != 3:
            raise ValueError
    
    def discriminant(self):
        return self.coefficients[1]**2 - 4*self.coefficients[0]*self.coefficients[2]
    
    def roots(self):
        return np.roots(self.coefficients)
    
    def roots2(self):
        a, b, c = self.coefficients
        disc = self.discriminant()
        
        if disc >= 0:
            solution_1 = (-b + np.sqrt(disc)) / (2 * a)
            solution_2 = (-b - np.sqrt(disc)) / (2 * a)
        else:
            solution_1 = (-b + 1j * np.sqrt(-disc)) / (2 * a)
            solution_2 = (-b - 1j * np.sqrt(-disc)) / (2 * a)
        
        return solution_1, solution_2


Now we test the code above

In [2]:
import numpy as np

# Define test polynomials
linear_poly = LinearPolynomial([2, 3])  # Represents 2x + 3
quadratic_poly_real = QuadraticPolynomial([1, -3, 2])  # Represents x^2 - 3x + 2
quadratic_poly_complex = QuadraticPolynomial([1, 1, 1])  # Represents x^2 + x + 1
general_poly = Polynomial([1, 0, -4])  # Represents x^2 - 4

# --- Testing __call__ ---
assert linear_poly(2) == 7, f"Expected 7, got {linear_poly(2)}"
assert quadratic_poly_real(3) == 2, f"Expected 2, got {quadratic_poly_real(3)}"
assert general_poly(2) == 0, f"Expected 0, got {general_poly(2)}"

# --- Testing call_2 ---
assert linear_poly.call_2(2) == 7, f"Expected 7, got {linear_poly.call_2(2)}"
assert quadratic_poly_real.call_2(3) == 2, f"Expected 2, got {quadratic_poly_real.call_2(3)}"
assert general_poly.call_2(2) == 0, f"Expected 0, got {general_poly.call_2(2)}"

# --- Testing __repr__ ---
assert str(linear_poly) == "2x+3", f"Expected '2x+3', got {str(linear_poly)}"
assert str(quadratic_poly_real) == "1x^2-3x+2", f"Expected '1x^2-3x+2', got {str(quadratic_poly_real)}"
assert str(quadratic_poly_complex) == "1x^2+1x+1", f"Expected '1x^2+1x+1', got {str(quadratic_poly_complex)}"
assert str(general_poly) == "1x^2-4", f"Expected '1x^2-4', got {str(general_poly)}"

# --- Testing Differentiation ---
assert str(differentiate(general_poly)) == "2x", f"Expected '2x', got {str(differentiate(general_poly))}"
assert str(differentiate(quadratic_poly_real)) == "2x-3", f"Expected '2x-3', got {str(differentiate(quadratic_poly_real))}"

# --- Testing Integration ---
assert str(integrate(general_poly)) == "0.3333333333333333x^3-4.0x", f"Expected '0.3333333333333333x^3-4.0x', got {str(integrate(general_poly))}"
assert str(integrate(quadratic_poly_real)) == "0.3333333333333333x^3-1.5x^2+2.0x", f"Expected '0.3333333333333333x^3-1.5x^2+2.0x', got {str(integrate(quadratic_poly_real))}"

# --- Testing __add__ ---
add_result = linear_poly + quadratic_poly_real
assert np.array_equal(add_result, [1, -1, 5]), f"Expected [1, -1, 5], got {add_result}"

# --- Testing add_2 ---
add_result_2 = linear_poly.add_2(quadratic_poly_real)
assert str(add_result_2) == "1x^2-1x+5", f"Expected '1x^2-1x+5', got {str(add_result_2)}"

# --- Testing __sub__ ---
sub_result = quadratic_poly_real - linear_poly
assert np.array_equal(sub_result, [1, -5, -1]), f"Expected [1, -5, -1], got {sub_result}"

# --- Testing sub_2 ---
sub_result_2 = quadratic_poly_real.sub_2(linear_poly)
assert str(sub_result_2) == "1x^2-5x-1", f"Expected '1x^2-5x-1', got {str(sub_result_2)}"

# --- Testing __mul__ ---
mul_result = linear_poly * quadratic_poly_real
assert np.array_equal(mul_result, [2, -3, -5, 6]), f"Expected [2, -3, -5, 6], got {mul_result}"

# --- Testing mul_2 ---
mul_result_2 = linear_poly.mul_2(quadratic_poly_real)
assert str(mul_result_2) == "2x^3-3x^2-5x+6", f"Expected '2x^3-3x^2-5x+6', got {str(mul_result_2)}"

# --- Testing Discriminant for QuadraticPolynomial ---
assert quadratic_poly_real.discriminant() == 1, f"Expected 1, got {quadratic_poly_real.discriminant()}"
assert quadratic_poly_complex.discriminant() == -3, f"Expected -3, got {quadratic_poly_complex.discriminant()}"

# --- Testing Roots (np.roots) ---
np_roots_real = quadratic_poly_real.roots()
assert np.allclose(np_roots_real, [2.0, 1.0]), f"Expected [2.0, 1.0], got {np_roots_real}"

np_roots_complex = quadratic_poly_complex.roots()
expected_complex_roots = [-0.5 + 0.8660254j, -0.5 - 0.8660254j]
assert np.allclose(np_roots_complex, expected_complex_roots), f"Expected {expected_complex_roots}, got {np_roots_complex}"

# --- Testing Custom Roots (roots2) ---
custom_roots_real = quadratic_poly_real.roots2()
assert np.allclose(custom_roots_real, (2.0, 1.0)), f"Expected (2.0, 1.0), got {custom_roots_real}"

custom_roots_complex = quadratic_poly_complex.roots2()
assert np.allclose(custom_roots_complex, expected_complex_roots), f"Expected {expected_complex_roots}, got {custom_roots_complex}"

# --- Testing get_roots ---
assert np.allclose(general_poly.get_roots(), [2.0, -2.0]), f"Expected [2.0, -2.0], got {general_poly.get_roots()}"

# --- Testing from_roots ---
poly_from_roots = Polynomial.from_roots(Polynomial, [2, -2])
assert str(poly_from_roots) == "1.0x^2-4.0", f"Expected '1.0x^2-4.0', got {str(poly_from_roots)}"

# --- Testing slope for LinearPolynomial ---
assert linear_poly.slope() == 3, f"Expected slope 3, got {linear_poly.slope()}"

# --- Edge Cases for Invalid Initialization ---
try:
    invalid_linear_poly = LinearPolynomial([1])  # Should raise a ValueError
except ValueError:
    print("Caught ValueError as expected for invalid LinearPolynomial (not enough coefficients).")

try:
    invalid_quadratic_poly = QuadraticPolynomial([1, 0])  # Should raise a ValueError
except ValueError:
    print("Caught ValueError as expected for invalid QuadraticPolynomial (not enough coefficients).")

print("All tests passed successfully.")


We are about to call __add__.
The result is [ 1 -1  5]
We are about to call add_2.
The result is 1x^2-1x+5
We are about to call __mul__.
The result is [ 2 -3 -5  6]
We are about to call mul_2.
The result is 2x^3-3x^2-5x+6
Caught ValueError as expected for invalid LinearPolynomial (not enough coefficients).
Caught ValueError as expected for invalid QuadraticPolynomial (not enough coefficients).
All tests passed successfully.
